## CMA-ES


**Disclaimer:** We make use of the implementation available at [PyPi](https://pypi.org/project/cma/) <cite data-cite="pycma"></cite> published by the author Nikolaus Hansen under the BSD license. 


CMA-ES which was proposed in <cite data-cite="cmaes"></cite>. Moreover, a comparing review can be found in <cite data-cite="cmaes-review"></cite>. 
CMA-ES stands for covariance matrix adaptation evolution strategy. Evolution strategies (ES) are stochastic, derivative-free methods for numerical optimization of non-linear or non-convex continuous optimization problems. They belong to the class of evolutionary algorithms and evolutionary computation. An evolutionary algorithm is broadly based on the principle of biological evolution, namely the repeated interplay of variation (via recombination and mutation) and selection: in each generation (iteration) new individuals (candidate solutions) are generated by variation, usually in a stochastic way, of the current parental individuals. Then, some individuals are selected to become the parents in the next generation based on their fitness or objective function value 
$f(x)$. Like this, over the generation sequence, individuals with better and better $f$-values are generated.
(excerpt from [Wikipedia](https://en.wikipedia.org/wiki/CMA-ES)).

### Example

In [1]:
from pymoo.algorithms.so_cmaes import CMAES
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("sphere")

algorithm = CMAES()

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print(f"Best solution found: \nX = {res.X}\nF = {res.X}\nCV= {res.CV}")

Best solution found: 
X = [0.49999997 0.49999994 0.5        0.49999996 0.5        0.5
 0.49999996 0.5        0.50000003 0.50000004]
F = [0.49999997 0.49999994 0.5        0.49999996 0.5        0.5
 0.49999996 0.5        0.50000003 0.50000004]
CV= [0.]


CMA-ES already has several stopping criteria implemented. However, as for other algorithms, the number of iterations or function evaluations can be directly passed to `minimize`.

In [2]:
res = minimize(problem,
               algorithm,
               ('n_iter', 10),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |      60 |  0.413583651 |  0.500000000 |  0.50000 |  0.50002 |  1.00005
    2 |      70 |  0.413583651 |  0.443532600 |  0.42922 |  0.44527 |  1.10591
    3 |      80 |  0.413583651 |  0.406582613 |  0.37335 |  0.41067 |  1.17817
    4 |      90 |  0.413583651 |  0.384570210 |  0.34673 |  0.39056 |  1.20631
    5 |     100 |  0.413583651 |  0.374835420 |  0.33287 |  0.41146 |  1.32408
    6 |     110 |  0.413583651 |  0.365537593 |  0.31713 |  0.40771 |  1.39290
    7 |     120 |  0.413583651 |  0.394854291 |  0.34417 |  0.42415 |  1.46804
    8 |     130 |  0.413583651 |  0.416364366 |  0.35289 |  0.44742 |  1.55281
    9 |     140 |  0.413583651 |  0.389834619 |  0.32470 |  0.42030 |  1.65435
   10 |     150 |  0.413583651 |  0.373800540 |  0.30839 |  0.39783 |  1.65274
Best solution found: 
X = [0.36998631 0.38142292 0.55648563 0.73116207 0.84201565 0.31814345
 0.41297808 0.4460369  0.7083321  0.84

In [3]:
res = minimize(problem,
               algorithm,
               ('n_evals', 50),
               seed=1,
               verbose=True)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

n_gen |  n_eval |     fopt     |    sigma     | min std  | max std  |   axis  
    1 |      60 |  0.230829022 |  0.500000000 |  0.50000 |  0.50002 |  1.00005
Best solution found: 
X = [0.39410806 0.31538908 0.76782696 0.38441356 0.42885057 0.43778444
 0.60280162 0.65186544 0.35727118 0.30633172]
F = [0.23082902]


Also, easily restarts can be used which are known to work very well on multi-modal functions. For instance, `Rastrigin` can be solved rather quickly by:

In [4]:
problem = get_problem("rastrigin")

algorithm = CMAES(restarts=10, restart_from_best=True)

res = minimize(problem,
               algorithm,
               ('n_evals', 2500),
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [ 4.50346187e-05 -7.31912594e-05]
F = [1.46514089e-06]


Our framework internally calls the `cma.fmin2` function. All parameters which can be used there either as a keyword argument or an option can also be passed to the `CMAES` constructor as well.
An example with a few selected `cma.fmin2` parameters is shown below:

In [5]:
import numpy as np
from pymoo.util.normalization import denormalize

# define an intitial point for the search
np.random.seed(1)
x0 = denormalize(np.random.random(problem.n_var), problem.xl, problem.xu)

algorithm = CMAES(x0=x0,
                 sigma=0.5,
                 restarts=2,
                 maxfevals=np.inf,
                 tolfun=1e-6,
                 tolx=1e-6,
                 restart_from_best=True,
                 bipop=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Best solution found: 
X = [-1.66760792e-07  7.76127578e-08]
F = [6.71107614e-12]


For more details about hyperparameters we refer to the software documentation of the `fmin2` in CMA-ES which can be found [here](http://cma.gforge.inria.fr/apidocs-pycma/cma.evolution_strategy.html#fmin2).
A quick explanation of possible parameters is also provided in the API documentation below.

### API